In [2]:

import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
        coalesce(vin, title + ' ' + dealership_name + ' - ' + DATE(scraped_date, 'start_month')) as vin
        , transmission
        , row_number() OVER (PARTITION BY vin, DATE(scraped_date, 'start of month')  ORDER BY scraped_date ASC) AS filter_row
      FROM
        inventory
      WHERE
        scraped_date >= 2022-03-01 and scraped_date <= 2022-08-29

'''
result = pd.read_sql_query(sql_query, conn)
result

,vin,transmission,filter_row


In [7]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

    with inventory_data as (
      SELECT
        coalesce(vin, title + ' ' + dealership_name + ' - ' + DATE(scraped_date, 'start_month')) as vin
        , make
        , DATE(scraped_date, 'start of month') as inventory_month
        , price
        , row_number() OVER (PARTITION BY vin, DATE(scraped_date, 'start of month')  ORDER BY scraped_date ASC) AS filter_row
      FROM
        inventory
      WHERE
        scraped_date >= '2022-01-01'
        and scraped_date <= '2022-08-31'
        and make in ('Ford')
    )
    select
      inventory_month,
      make,
      avg(price) as price
    from
      inventory_data
    where
      filter_row = 1
    group by
      inventory_month
      , make
    order by
      inventory_month DESC, make ASC;

'''
result = pd.read_sql_query(sql_query, conn)
result

,inventory_month,make,price
0,2022-08-01,Ford,32303.299435
1,2022-07-01,Ford,32002.654545
2,2022-05-01,Ford,30110.044118
3,2022-04-01,Ford,29628.801653
4,2022-03-01,Ford,28935.120567


In [4]:
# ## Targeted drop
# TABLE_NAME = 'inventory'

# #Connecting to sqlite
# conn = sqlite3.connect('./data/cars.db')

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Doping EMPLOYEE table if already exists
# cursor.execute(f"DELETE FROM {TABLE_NAME} WHERE make is null")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()